# 30_ranked_nominal — 순위형 + 명목형(요약)

In [ ]:
%run 00_config.ipynb
import pandas as pd, numpy as np

def wprop(series, w):
    s = series.dropna()
    w = w.loc[s.index]
    tot = w.sum()
    if tot<=0: return pd.Series(dtype=float)
    return (w.groupby(s).sum()/tot*100).sort_index()

df = pd.read_excel(DATA_PATH)
w = choose_weight(df)

# Ranked
rank = df.copy()[RANK_VARS]
def to_score(s, weights): return s.map(PRIORITY_WEIGHT)

for c in RANK_VARS:
    if c in rank.columns:
        rank[f"{c}_score"] = to_score(rank[c], PRIORITY_WEIGHT)

rows = []
for c in RANK_VARS:
    sc = f"{c}_score"
    if sc not in rank.columns: continue
    valid = rank[sc].notna()
    w_mean = np.average(rank.loc[valid, sc], weights=w.loc[valid]) if valid.any() else np.nan
    u_mean = rank[sc].mean()
    rows.append({"variable": c, "weighted_mean_score": round(float(w_mean),3) if pd.notna(w_mean) else np.nan,
                 "unweighted_mean_score": round(float(u_mean),3) if pd.notna(u_mean) else np.nan})

rank_summary = pd.DataFrame(rows).sort_values("variable")
rank_summary.to_csv(OUT_DIR/"ranked_summary.csv", index=False, encoding="utf-8-sig")

# Nominal(요약) — 간단 분포
nom_list = var_types["categorical-nominal"]["questions"]
rows = []
for col in nom_list:
    if col not in df.columns: continue
    ws = wprop(df[col], w)
    us = (df[col].value_counts(dropna=True, normalize=True)*100).round(2)
    for lvl, pct in ws.items():
        rows.append({"variable": col, "level": lvl, "weighted_%": round(pct,2)})
    for lvl, pct in us.items():
        rows.append({"variable": col, "level": lvl, "unweighted_%": pct})

nom_summary = pd.DataFrame(rows).groupby(["variable","level"], as_index=False).sum(numeric_only=True)\
                                .sort_values(["variable","level"])
nom_summary.to_csv(OUT_DIR/"categorical_nominal_summary_quick.csv", index=False, encoding="utf-8-sig")

rank_summary.head(10)
